<a href="https://colab.research.google.com/github/tiagosilveiraa/portifoliods/blob/main/Players_Recomendation_using_fifa22_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [291]:
import pandas as pd
import requests
import re
from decimal import Decimal
import numpy as np
from datetime import datetime
import time
from google.colab import data_table

In [292]:
data_table.disable_dataframe_formatter()
np.random.seed(777)

#1 - Loading dataset

In [293]:
df = pd.read_csv('https://github.com/tiagosilveiraa/portifoliods/raw/main/datasets/FIFA22_official_data.csv', sep=',')

In [294]:
df = df.rename(columns=lambda x: x.replace(" ", "_").lower())
df = df.rename(columns={'value': 'market_value'})

#2 - Data cleaning

##2.1 - Formatting Values

In [295]:
def str_to_currency(value):
  if(type(value) == str):
    try:    
        x = re.sub(r'[^\d.]', '', value)
        x = Decimal(x) 
    except Exception as e:
      raise e
    else:
      if x == None:
        return np.NaN
      if 'K' in value:
        return x * 1000
      elif 'M' in value:
        return x * 1000000

In [296]:
df['release_clause'] = df['release_clause'].apply(str_to_currency)
df['wage'] = df['wage'].apply(str_to_currency)
df['market_value'] = df['market_value'].apply(str_to_currency)

In [297]:
def to_date(date):     
    try:  
      date_size = len(str(date))   
      formatted_date = ''        
      if date_size > 4:
        formatted_date = datetime.strptime(date, '%b %d, %Y')      
      elif date_size == 4:
        formatted_date = datetime(int(date), 6, 30) 
      else:
        next_year = datetime.today().year + 1
        next_season_end = datetime(next_year, 12,31)
        formatted_date = next_season_end
    except Exception as e:
      raise e
    else:    
      return time.mktime(formatted_date.timetuple())

In [298]:
df['contract_valid_until'] = df['contract_valid_until'].apply(to_date)
df['joined'] = df['joined'].apply(to_date)

In [299]:
def remove_html(value):
  if(type(value) == str):
    try:
      x = re.sub(r'<.{0,100}">|<.{0,10}>','', value)
    except Exception as e:
      raise e
    else:
      return x

In [300]:
df['position'] = df['position'].apply(remove_html)
df['loaned_from'] = df['loaned_from'].apply(remove_html)

In [301]:
def str_to_number(value):
  if value:
    try:
      x = re.sub(r'[^\d.]', '', value)
      x = Decimal(x) 
    except Exception as e:
      raise e
    else:
      return x

In [302]:
df['height'] = df['height'].apply(str_to_number)
df['weight'] = df['weight'].apply(str_to_number)

##2.2 - Changing datatypes

In [303]:
df['wage'] = pd.to_numeric(df['wage'])
df['market_value'] = pd.to_numeric(df['market_value'])
df['height'] = pd.to_numeric(df['height'])
df['weight'] = pd.to_numeric(df['weight'])
df['release_clause'] = pd.to_numeric(df['release_clause'])

##2.2 - Filling NA values

In [304]:
df.columns[df.isna().any()].tolist()

['club',
 'market_value',
 'wage',
 'body_type',
 'real_face',
 'position',
 'jersey_number',
 'loaned_from',
 'volleys',
 'curve',
 'agility',
 'balance',
 'jumping',
 'interceptions',
 'positioning',
 'vision',
 'composure',
 'marking',
 'slidingtackle',
 'release_clause',
 'defensiveawareness']

Removing players without Market Value, Club, Wage, Release Clause Value, Joined Date because we can't infer these information.

In [305]:
df.dropna(subset = ['market_value', 'club', 'wage', 'release_clause', 'joined'], inplace=True)

Here we are assuming that every player that has not the information of the club that Loaned him is not loaned.

In [306]:
df['loaned_from'].fillna('NL', inplace=True)

The attribute Marking was substituted for the attribute DefensiveAwareness in Fifa 22 so, unified the two columns

In [307]:
df.loc[df['defensiveawareness'].isna() == True, ['defensiveawareness']] = df.loc[df['defensiveawareness'].isna() == True]['marking']

In [308]:
df.drop(columns=['marking'], inplace = True)

In [309]:
df.loc[df['position'].isin(['RES', 'SUB']), ['position']] = df.loc[df['position'].isin(['RES', 'SUB'])]['best_position']

## 2.3 - Exploding attributes

The attribute Work Rate can be divided in Deffensive Workrate and Attacking Workrate, in this dataset both are provided in the same column so, i had to split it into two different columns

In [310]:
awr = []
dwr = []
for i, w in enumerate(df['work_rate'].tolist()): 
  if(type(w) == str): 
    opt = w.replace(' ', '').split('/')
    awr.append(opt[0])
    dwr.append(opt[1])
  else:
    awr.append(np.nan)
    dwr.append(np.nan)

In [311]:
df['attacking_wr'] = awr
df['defensive_wr'] = dwr

#3 - Data Modeling

Dropping unecessary columns to the model

In [312]:
X = df.drop(columns=['photo', 'flag','jersey_number', 'joined', 'club_logo', 'work_rate', 'name', 'special'])

In [313]:
#X.set_index('id', inplace=True)

## 3.1 - Applying encodings

In [314]:
from sklearn.preprocessing import OneHotEncoder

In [315]:
X.select_dtypes('object').columns

Index(['nationality', 'club', 'preferred_foot', 'body_type', 'real_face',
       'position', 'loaned_from', 'best_position', 'attacking_wr',
       'defensive_wr'],
      dtype='object')

In [316]:
wr_order = ['Low', 'Medium','High',]
X['attacking_wr'] =  X['attacking_wr'].astype('category')
X['attacking_wr'] = X['attacking_wr'].cat.reorder_categories(wr_order).cat.codes
X['defensive_wr'] =  X['defensive_wr'].astype('category')
X['defensive_wr'] = X['defensive_wr'].cat.reorder_categories(wr_order).cat.codes

In [317]:
rf_order = ['No', 'Yes']
X['real_face'] = X['real_face'].astype('category')
X['real_face'] = X['real_face'].cat.reorder_categories(rf_order).cat.codes

In [318]:
one_nation = OneHotEncoder()
#one_club = OneHotEncoder()
one_foot = OneHotEncoder()
one_body_type = OneHotEncoder()
one_position = OneHotEncoder()
#one_loaned_from = OneHotEncoder()
one_best_position = OneHotEncoder()

In [319]:
enc_nation = one_nation.fit_transform(X[['nationality']])
#enc_club = one_club.fit_transform(X[['club']])
enc_foot = one_foot.fit_transform(X[['preferred_foot']])
enc_body_type = one_body_type.fit_transform(X[['body_type']])
enc_position = one_position.fit_transform(X[['position']])
#enc_loaned_from = one_loaned_from.fit_transform(X[['loaned_from']])
enc_best_position = one_best_position.fit_transform(X[['best_position']])

In [320]:
X[one_nation.categories_[0]] = enc_nation.toarray()
#X[one_club.categories_[0]] = enc_club.toarray()
X[one_foot.categories_[0]] = enc_foot.toarray()
X[one_body_type.categories_[0]] = enc_body_type.toarray()
X[[f'default_{x}' for x in one_position.categories_[0]]] = enc_position.toarray()
X[[f'best_{x}' for x in one_best_position.categories_[0]]] = enc_best_position.toarray()
#X[[f'loaned_{x}' for x in one_loaned_from.categories_[0]]] = enc_loaned_from.toarray()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


In [321]:
X.drop(columns=['nationality', 'club', 'preferred_foot', 'body_type', 'position',
       'loaned_from', 'best_position'], inplace=True)

##3.2 - Rescaling values

In [322]:
from sklearn.preprocessing import MinMaxScaler

In [323]:
scaler = MinMaxScaler()

In [324]:
X_scaled = pd.DataFrame()

In [325]:
X_scaled[X.columns.values] =  scaler.fit_transform(X[X.columns.values])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


In [326]:
X_scaled.index = X['id']

In [327]:
X_scaled.drop(columns=['id'], inplace=True)

##3.3 - Creating model

In [340]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute', n_neighbors=5)
model.fit(X_scaled.values)

NearestNeighbors(algorithm='brute')

# 4 - Presenting results

In [329]:
from ipywidgets import widgets, interact
from IPython.display import display

In [330]:
dp_players = df[['name','id']].sort_values(by=['name']).values.tolist()

In [331]:
def get_similars(Player):
  distance, similar_players_index = model.kneighbors(X_scaled.loc[Player].values.reshape(1,-1))  
  similar_players = pd.DataFrame()
  similar_players = df.loc[df.index[similar_players_index[0]]]
  similar_players['distances'] = distance[0]
  return similar_players[['name', 'club', 'overall','distances']]

In [341]:
interact(get_similars, Player=dp_players);

interactive(children=(Dropdown(description='Player', options=(['18\xa0A. Aquilani', 103935], ['18\xa0A. Araos'…

#5 - Exporting results

In [351]:
from google.colab import files
from joblib import dump

In [353]:
df.to_json('players.json', orient='records')
files.download('players.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [354]:
dump(model,'model.joblib')

['model.joblib']